# Installation Notes for K8s and KF on GCP GKE

This is WORK IN PROGRESS. Not finished.

## Instructions: 
- https://www.kubeflow.org/docs/distributions/gke/deploy/overview/

## Set up GCP project and API scopes
https://www.kubeflow.org/docs/distributions/gke/deploy/project-setup/
```
gcloud services enable \
  compute.googleapis.com \
  container.googleapis.com \
  iam.googleapis.com \
  servicemanagement.googleapis.com \
  cloudresourcemanager.googleapis.com \
  ml.googleapis.com \
  iap.googleapis.com \
  sqladmin.googleapis.com \
  meshconfig.googleapis.com

  # Cloud Build API is optional, you need it if using Fairing.
  # gcloud services enable cloudbuild.googleapis.com
```

## # Initialize your project to prepare it for Anthos Service Mesh installation
```
PROJECT_ID=<GCP Project ID Number>

# Create a temporary cluster (workaround for installing Anthos Mesh if there are no other clusters)
# https://github.com/kubeflow/website/issues/2121
gcloud beta container clusters create tmp-cluster \
  --release-channel regular \
  --workload-pool=${PROJECT_ID}.svc.id.goog \
  --zone=europe-west1-b

curl --request POST \
  --header "Authorization: Bearer $(gcloud auth print-access-token)" \
  --data '' \
  https://meshconfig.googleapis.com/v1alpha1/projects/${PROJECT_ID}:initialize

# Remove the temporary cluster
gcloud beta container clusters delete tmp-cluster --zone=europe-west1-b
```

## Set up OAuth client
https://www.kubeflow.org/docs/distributions/gke/deploy/oauth-setup/
```
skipped for the dev/test deployment
```

## Deploy Management cluster
https://www.kubeflow.org/docs/distributions/gke/deploy/management-setup/
The management cluster is used to run Cloud Config Connector. Cloud Config Connector is a Kubernetes addon that allows you to manage Google Cloud resources through Kubernetes.
```
#gcloud components install kubectl kpt anthoscli beta
sudo apt-get install kubectl google-cloud-sdk-kpt google-cloud-sdk google-cloud-sdk

#gcloud components update
sudo apt-get update && sudo apt-get --only-upgrade install google-cloud-sdk-config-connector google-cloud-sdk-app-engine-go google-cloud-sdk google-cloud-sdk-anthos-auth google-cloud-sdk-skaffold google-cloud-sdk-app-engine-python google-cloud-sdk-cloud-build-local google-cloud-sdk-cbt google-cloud-sdk-app-engine-grpc google-cloud-sdk-minikube google-cloud-sdk-pubsub-emulator google-cloud-sdk-local-extract google-cloud-sdk-app-engine-python-extras google-cloud-sdk-spanner-emulator google-cloud-sdk-datastore-emulator google-cloud-sdk-kpt google-cloud-sdk-bigtable-emulator google-cloud-sdk-firestore-emulator google-cloud-sdk-app-engine-java google-cloud-sdk-kubectl-oidc kubectl google-cloud-sdk-datalab

# kpt v1.0.0-beta.1 or above doesn’t work due to a known issue
# downgrade to https://github.com/GoogleContainerTools/kpt/releases/tag/v0.39.2
kpt version

# https://cloud.google.com/service-mesh/docs/environment-setup
curl -L https://github.com/GoogleContainerTools/kpt/releases/download/v0.39.2/kpt_linux_amd64 > kpt_0_39_2
chmod +x kpt_0_39_2
alias kpt="$(readlink -f kpt_0_39_2)"

# link powinien byc w .bashrc, ale to nie dziala gdy wykonuje sktypr kpt-set.sh
# alias kpt="~/kpt_0_39_2"
# vi i esc :wq
sudo mv /usr/bin/kpt /usr/bin/kpt_bkp
sudo cp ~/kpt_0_39_2 /usr/bin/kpt

kpt version

# if older version exists
rm -rf ~/kustomize

# Detect your OS and download corresponding latest Kustomize binary
curl -s "https://raw.githubusercontent.com/kubernetes-sigs/kustomize/master/hack/install_kustomize.sh"  | bash

# Add the kustomize package to your $PATH env variable
sudo mv ~/kustomize /usr/local/bin/kustomize

kustomize version

sudo wget https://github.com/mikefarah/yq/releases/download/3.4.1/yq_linux_amd64 -O /usr/bin/yq && sudo chmod +x /usr/bin/yq

# The Kubeflow deployment process is not compatible with yq v4 or later
yq --version

Fetch kubeflow/gcp-blueprints package

# if older version exists
rm -rf ~/gcp-blueprints

# Check out Kubeflow v1.3.1 blueprints
kpt pkg get https://github.com/kubeflow/gcp-blueprints.git@v1.3.1 gcp-blueprints
cd gcp-blueprints
cd management

---
Configure Environment Variables
MGMT_PROJECT=<GCP Project ID Number>
MGMT_DIR=~/gcp-blueprints/management/
MGMT_NAME=test-kf-mgr
LOCATION=europe-west1-b

---
Configure kpt setter values
kpt cfg set -R . name "${MGMT_NAME}"
kpt cfg set -R . gcloud.core.project "${MGMT_PROJECT}"
kpt cfg set -R . location "${LOCATION}"

kpt cfg list-setters .

---
Deploy Management Cluster

# Deploy the management cluster by applying cluster resources
make apply-cluster

# Create a kubectl context for the management cluster, it will be named ${MGMT_NAME}
make create-context

# Install the Cloud Config Connector
# This step:
# - Installs Config Connector in your cluster, and
# - Creates the Google Cloud service account ${MGMT_NAME}-cnrm-system@${MGMT_PROJECT}.iam.gserviceaccount.com.
```

## Deploy Kubeflow cluster
https://www.kubeflow.org/docs/distributions/gke/deploy/deploy-cli/
```
# https://stedolan.github.io/jq/
sudo apt install jq

# Fetch kubeflow/gcp-blueprints and upstream packages
cd ~/gcp-blueprints/kubeflow
bash ./pull-upstream.sh

# Environment Variables
# Log in to gcloud. You only need to run this command once
# How-to: https://stackoverflow.com/questions/36273455/where-does-the-verification-code-for-gcloud-auth-login-come-from
gcloud auth login

# Review and fill all the environment variables in gcp-blueprints/kubeflow/env.sh
vi env.sh
# i esc :wq
KF_PROJECT=<GCP Project ID>
KF_PROJECT_NUMBER=<GCP Project ID Number>
ADMIN_EMAIL=<E-MAIL>
MGMT_NAME=test-kf-mgr

LOCATION=europe-west1-b
REGION=europe-west1
ZONE=europe-west1-b

# Set environment variables with OAuth Client ID and Secret for IAP (optional)
export CLIENT_ID=<Your CLIENT_ID>
export CLIENT_SECRET=<Your CLIENT_SECRET>

# Must be run on the same terminal session
# source env.sh
# bash kpt-set.sh
# ...
# make apply

source env.sh
gcloud projects describe --format='value(projectNumber)' "${KF_PROJECT}"

# Configure Kubeflow
bash ./kpt-set.sh

#kpt cfg set -R . mgmt-ctxt "${MGMT_NAME}"

kpt cfg list-setters .
kpt cfg list-setters common/managed-storage
kpt cfg list-setters apps/pipelines

# Management cluster config
# Choose the management cluster context
kubectl config use-context "${MGMTCTXT}"

# Create a namespace in your management cluster for the Kubeflow project if you haven’t done so
kubectl create namespace "${KF_PROJECT}"
```

## Authorize Cloud Config Connector for each Kubeflow project
n the Management cluster deployment we created the Google Cloud service account ${MGMT_NAME}-cnrm-system@${MGMT_PROJECT}.iam.gserviceaccount.com this is the service account that Config Connector will use to create any Google Cloud resources. If your Management cluster and Kubeflow cluster live in different projects, you need to grant this Google Cloud service account sufficient privileges to create the desired resources in Kubeflow project.

The easiest way to do this is to grant the Google Cloud service account owner permissions on one or more projects.
```
# <the project where you deploy your management cluster>
MGMT_PROJECT=<GCP Project ID>

# Redirect to management directory and configure kpt setter
pushd "../management"
kpt cfg set -R . name "${MGMT_NAME}"
kpt cfg set -R . gcloud.core.project "${MGMT_PROJECT}"
kpt cfg set -R . managed-project "${KF_PROJECT}"

# Update the policy
gcloud beta anthos apply ./managed-project/iam.yaml

# Return to gcp-blueprints/kubeflow
popd
```

## Deploy Kubeflow
```
make apply
```